## Data transformation - COVID 19 dataset

merge two tables

In [1]:
%%bigquery
select * from covid_19_staging.Winter_Cases limit 6

,state,country,last_update,confirmed,deaths,recovered,latitude,longitude
0,None,Italy,2020-01-31T08:15:53,2,0,0,None,None
1,None,United Arab Emirates,2020-02-22T14:53:02,13,0,4,None,None
2,None,Sweden,2020-02-01T02:13:26,1,0,0,None,None
3,None,Cambodia,1/31/2020 8:15,1,0,0,None,None
4,None,North Macedonia,2020-02-26T19:03:08,1,0,0,None,None
5,None,Sri Lanka,2020-02-08T03:43:03,1,0,1,None,None


In [8]:
%%bigquery
select * from covid_19_staging.Spring_Cases limit 6

,fips,admin2,state,country,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key
0,NaN,None,None,Chad,2020-03-31 23:43:43,15.4542,18.7322,7,0,0,7,Chad
1,NaN,None,None,Chad,2020-04-01 21:58:34,15.4542,18.7322,7,0,0,7,Chad
2,NaN,None,None,Chad,4/2/20 23:25,15.4542,18.7322,8,0,0,8,Chad
3,NaN,None,None,Chad,2020-04-03 22:46:20,15.4542,18.7322,8,0,0,8,Chad
4,NaN,None,None,Chad,4/4/20 23:34,15.4542,18.7322,9,0,0,9,Chad
5,NaN,None,None,Chad,2020-04-05 23:06:26,15.4542,18.7322,9,0,0,9,Chad


In [17]:
%%bigquery
select count(*) from covid_19_staging.Winter_Cases 
union all
select count(*) from covid_19_staging.Spring_Cases

,f0_
0,7617
1,120703


In [12]:
%%bigquery
create table covid_19_staging.all_cases as
select state, country, last_update, cast(latitude as String) as latitude , cast(longitude as String) as longitude, ifnull(confirmed,0) as confirmed, ifnull(deaths,0) as deaths, ifnull(recovered,0) as recovered from covid_19_staging.Winter_Cases 
union all
select state, country, last_update, latitude, longitude, ifnull(confirmed,0), ifnull(deaths,0), ifnull(recovered,0) from covid_19_staging.Spring_Cases


""


In [13]:
%%bigquery
select * from covid_19_staging.all_cases limit 6

,state,country,last_update,latitude,longitude,confirmed,deaths,recovered
0,Hubei,Mainland China,2020-02-02T23:43:02,None,None,11177,350,295
1,Hubei,Mainland China,1/25/20 17:00,None,None,761,40,32
2,Michigan,US,2020-04-17 23:30:52,None,None,514,15,0
3,New Jersey,US,2020-04-21 23:30:50,None,None,3073,233,0
4,Arizona,US,2020-04-09 23:02:37,None,None,0,18,0
5,Maryland,US,2020-04-26 02:31:04,None,None,0,124,0


In [14]:
%%bigquery
select count(*) from covid_19_staging.all_cases

,f0_
0,128320


Standardize timestamps

In [16]:
%%bigquery
create table covid_19_staging.cases_sql_1 as
select *
from covid_19_staging.all_cases
where strpos(last_update, '/') > 0

""


In [17]:
%%bigquery
select count(*) from covid_19_staging.cases_sql_1

,f0_
0,12054


In [18]:
%%bigquery
create table covid_19_staging.cases_sql_2 as
select *
from covid_19_staging.all_cases
where strpos(last_update, '-') > 0

""


In [19]:
%%bigquery
select count(*) from covid_19_staging.cases_sql_2

,f0_
0,116266


In [23]:
%%bigquery
select last_update, length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) as year_length
from
(select distinct last_update
from covid_19_staging.cases_sql_1)
limit 10

,last_update,year_length
0,1/28/20 23:00,2
1,1/30/20 16:00,2
2,1/29/20 19:30,2
3,1/26/20 16:00,2
4,1/25/20 17:00,2
5,1/23/20 17:00,2
6,1/31/2020 10:37,4
7,1/24/20 17:00,2
8,1/27/20 23:59,2
9,4/2/20 23:25,2


In [24]:
%%bigquery
select last_update, parse_datetime('%m/%d/%y %H:%M', last_update) as last_update_datetime
from
(select distinct last_update
from covid_19_staging.cases_sql_1)
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 2
limit 10

,last_update,last_update_datetime
0,1/28/20 23:00,2020-01-28 23:00:00
1,1/30/20 16:00,2020-01-30 16:00:00
2,1/29/20 19:30,2020-01-29 19:30:00
3,1/26/20 16:00,2020-01-26 16:00:00
4,1/25/20 17:00,2020-01-25 17:00:00
5,1/23/20 17:00,2020-01-23 17:00:00
6,1/24/20 17:00,2020-01-24 17:00:00
7,1/27/20 23:59,2020-01-27 23:59:00
8,4/2/20 23:25,2020-04-02 23:25:00
9,4/6/20 23:21,2020-04-06 23:21:00


In [36]:
%%bigquery
create or replace table covid_19_staging.cases_sql_3 as
select state, country, parse_datetime('%m/%d/%y %H:%M', last_update) as last_update, confirmed, deaths, recovered
from covid_19_staging.cases_sql_1
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 2
union all
select state, country, parse_datetime('%m/%d/%Y %H:%M', last_update) as last_update, confirmed, deaths, recovered
from covid_19_staging.cases_sql_1
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 4

""


In [56]:
%%bigquery
select count(*) from covid_19_staging.cases_sql_3 

,f0_
0,12054


In [40]:
%%bigquery
create or replace table covid_19_staging.cases_sql_4 as
select state,country, cast(last_update as DATETIME) as last_update ,confirmed,deaths,recovered from covid_19_staging.cases_sql_2
union all
select * from covid_19_staging.cases_sql_3

""


In [55]:
%%bigquery
select count(*) from covid_19_staging.cases_sql_4

,f0_
0,128320


In [50]:
%%bigquery
select state,country, last_update, count(*) as duplicate_events
from covid_19_staging.cases_sql_4
group by state,country, last_update
having count(*) > 1
order by count(*) desc

,state,country,last_update,duplicate_events
0,Texas,US,2020-03-30 22:52:00,254
1,Texas,US,2020-05-09 02:32:35,220
2,Texas,US,2020-05-08 02:32:32,219
3,Texas,US,2020-05-07 02:32:28,218
4,Texas,US,2020-05-06 02:32:31,218
...,...,...,...,...
3080,United States Virgin Islands,US,2020-03-18 00:13:13,2
3081,"Omaha, NE (From Diamond Princess)",US,2020-02-22 23:03:09,2
3082,"Travis, CA (From Diamond Princess)",US,2020-02-22 17:13:24,2
3083,"Lackland, TX (From Diamond Princess)",US,2020-02-22 17:13:24,2


In [61]:
%%bigquery
create or replace table covid_19_staging.cases_SQL_Final as
select distinct state,country, last_update, confirmed, deaths, recovered
from
(select *, 
rank() over (partition by state,country,last_update order by confirmed desc) as rank
from covid_19_staging.cases_sql_4)
where rank = 1

""


In [62]:
%%bigquery
select count(*) from covid_19_staging.cases_SQL_Final

,f0_
0,17322


In [65]:
%%bigquery
select distinct * from covid_19_staging.cases_SQL_Final order by state, country, last_update

,state,country,last_update,confirmed,deaths,recovered
0,None,Azerbaijan,2020-02-28 15:03:26,1,0,0
1,None,Afghanistan,2020-02-24 23:33:02,1,0,0
2,None,Afghanistan,2020-03-08 04:53:03,4,0,0
3,None,Afghanistan,2020-03-10 12:33:02,5,0,0
4,None,Afghanistan,2020-03-11 10:13:12,7,0,0
...,...,...,...,...,...,...
17317,Zhejiang,Mainland China,2020-03-06 10:23:03,1215,1,1147
17318,Zhejiang,Mainland China,2020-03-07 09:03:05,1215,1,1154
17319,Zhejiang,Mainland China,2020-03-08 09:03:04,1215,1,1161
17320,Zhejiang,Mainland China,2020-03-09 09:43:03,1215,1,1176


In [1]:
%%bigquery
create or replace table covid_19_staging.cases_SQL_Final_Date as
select state,country, cast(last_update as date) as date,confirmed,deaths,recovered
from covid_19_staging.cases_SQL_Final

""


### Data transformation - Apple mobility dataset

In [67]:
%%bigquery
select * from apple_mobility_staging.mobility_table limit 6

,geo_type,region,transportation_type,alternative_name,_2020_01_13,_2020_01_14,_2020_01_15,_2020_01_16,_2020_01_17,_2020_01_18,...,_2020_04_28,_2020_04_29,_2020_04_30,_2020_05_01,_2020_05_02,_2020_05_03,_2020_05_04,_2020_05_05,_2020_05_06,_2020_05_07
0,country/region,Austria,driving,Österreich,100.0,101.14,104.24,112.21,117.23,117.22,...,62.63,62.29,64.44,61.20,62.46,66.38,74.19,74.04,78.21,80.89
1,country/region,Austria,walking,Österreich,100.0,101.55,105.59,112.24,123.36,131.05,...,47.74,40.06,46.55,46.62,49.18,46.38,54.51,53.07,56.95,61.40
2,country/region,Belgium,driving,België,100.0,101.19,107.49,107.67,117.38,119.32,...,51.69,54.59,54.06,48.03,50.82,54.76,61.76,64.06,66.62,68.02
3,country/region,Belgium,transit,België,100.0,98.67,105.01,105.87,113.55,110.73,...,25.95,26.25,28.09,27.73,27.10,33.19,32.50,32.93,35.82,36.10
4,country/region,Belgium,walking,België,100.0,101.46,110.44,118.86,139.10,174.99,...,42.77,50.36,51.45,51.80,60.18,74.43,61.50,64.98,71.65,73.34
5,country/region,Brazil,driving,Brasil,100.0,99.71,100.90,101.88,113.69,114.38,...,54.57,56.02,64.63,55.37,58.58,42.66,55.50,59.15,61.80,60.70


In [68]:
%%bigquery
select count(*) from apple_mobility_staging.mobility_table

,f0_
0,1158
